In [1]:
import sys, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

# print(os.listdir("./"))
pd.set_option('display.max_columns', None)

from collections import Counter 
from collections import defaultdict
import itertools


In [2]:
df = pd.read_csv('./survey_results_public.csv')
df.head(5)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,Strongly agree,Strongly disagree,Agree,Agree,Disagree,Agree,"I'm not actively looking, but I am open to new...",0.0,Not applicable/ never,Very important,Very important,Important,Very important,Very important,Very important,Important,Very important,Very important,Very important,Very important,Very important,Somewhat important,Not very important,Somewhat important,Stock options; Vacation/days off; Remote options,Yes,Other,NaN,NaN,Important,Important,Important,Somewhat important,Important,Not very important,Not very important,Not at all important,Somewhat important,Very important,NaN,NaN,Tabs,NaN,Online course; Open source contributions,NaN,NaN,NaN,6:00 AM,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS,Atom; Xcode,Turn on some music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat satisfied,Not very satisfied,Not at all satisfied,Very satisfied,Satisfied,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stac...,9.0,Desktop; iOS app,At leas

In [3]:
df.shape

(19102, 154)

In [4]:
schDf = pd.read_csv('./survey_results_schema.csv')
schDf.head(10)

,Column,Question
0,Respondent,Respondent ID number
1,Professional,Which of the following best describes you?
2,ProgramHobby,Do you program as a hobby or contribute to ope...
3,Country,In which country do you currently live?
4,University,"Are you currently enrolled in a formal, degree..."
5,EmploymentStatus,Which of the following best describes your cur...
6,FormalEducation,Which of the following best describes the high...
7,MajorUndergrad,Which of the following best describes your mai...
8,HomeRemote,How often do you work from home or remotely?
9,CompanySize,"In terms of the number of employees, how large..."


In [5]:
schDf.shape

(154, 2)

In [6]:
df.describe()

,Respondent,CareerSatisfaction,JobSatisfaction,HoursPerWeek,StackOverflowSatisfaction,Salary,ExpectedSalary
count,19102.000000,16102.000000,15253.000000,7721.000000,13452.000000,5009.000000,818.000000
mean,9551.500000,7.343932,7.000721,3.685922,8.357865,58341.398571,32231.521687
std,5514.416757,1.919330,2.128648,7.330691,1.405546,40753.384986,28543.877274
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4776.250000,6.000000,6.000000,0.000000,8.000000,27956.989247,4716.459735
50%,9551.500000,8.000000,7.000000,1.000000,8.000000,51612.903226,29784.946237
75%,14326.750000,9.000000,8.000000,3.000000,9.000000,83333.333333,50000.000000
max,19102.000000,10.000000,10.000000,40.000000,10.000000,195000.000000,120000.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19102 entries, 0 to 19101
Columns: 154 entries, Respondent to ExpectedSalary
dtypes: float64(6), int64(1), object(147)
memory usage: 22.4+ MB


In [8]:
schDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 2 columns):
Column      154 non-null object
Question    154 non-null object
dtypes: object(2)
memory usage: 2.5+ KB


# What are trends on tech tools?

In [9]:
df[["HaveWorkedLanguage", "WantWorkLanguage", "HaveWorkedFramework", "WantWorkFramework", "HaveWorkedDatabase", "WantWorkDatabase", "HaveWorkedPlatform", "WantWorkPlatform"]].dropna(how='all')

,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform
0,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS
1,JavaScript; Python; Ruby; SQL,Java; Python; Ruby; SQL,.NET Core,.NET Core,MySQL; SQLite,MySQL; SQLite,Amazon Web Services (AWS),Linux Desktop; Raspberry Pi; Amazon Web Servic...
2,Java; PHP; Python,C; Python; Rust,NaN,NaN,MySQL,NaN,NaN,NaN
3,Matlab; Python; R; SQL,Matlab; Python; R; SQL,React,Hadoop; Node.js; React,MongoDB; Redis; SQL Server; MySQL; SQLite,MongoDB; Redis; SQL Server; MySQL; SQLite,Windows Desktop; Linux Desktop; Mac OS; Amazon...,Windows Desktop; Linux Desktop; Mac OS; Amazon...
5,JavaScript; PHP; Rust,Clojure; Elixir; Erlang; Haskell; Rust; TypeSc...,NaN,NaN,MySQL,NaN,Linux Desktop; Amazon Web Services (AWS),Linux Desktop
6,Matlab; Python,JavaScript; Julia; Matlab; Python; R; SQL,NaN,Hadoop; Node.js,SQLite,MongoDB; SQL Server; PostgreSQL; SQLite,Windows Desktop,Arduino; Raspberry Pi
7,CoffeeScript; Clojure; Elixir; Erlang; Haskell...,Clojure; Elixir; Haskell; Scala,React,NaN,Redis; MySQL; PostgreSQL,NaN,Amazon Web Services (AWS),NaN
8,C#; JavaScript,F#; Go,AngularJS; Node.js; .NET Core,Hadoop; React; Spark,SQL Server,Cassandra; Redis,Android; Microsoft Azure,Serverless; Amazon Web Services (AWS); Microso...
9,Objective-C; Swift,C#; Java; Objective-C; Swift,NaN,.NET Core,PostgreSQL,NaN,iOS; Amazon Web Services (AWS),Android; iOS; Windows Phone; Windows Desktop
10,R; SQL,C#; R; SQL,NaN,NaN,SQL Server,SQL Server,NaN,NaN


In [10]:
def techTrend(field):
    new_df = defaultdict(int)
    c = field
    types= ["HaveWorked", "WantWork"]

    for col_type in types:

        each = df[col_type + field].dropna()
        each.shape[0]
        List = '; '.join(each.tolist()).split("; ")
        new_df[col_type + field] = List

        Df = pd.DataFrame.from_dict(Counter(List), orient='index').reset_index().set_index('index')
        Df.rename(columns={0: col_type}, inplace=True)

        if col_type=="HaveWorked":
            HaveWorkedDf = Df
            HaveWorked = each.shape[0]
        elif col_type=="WantWork":
            WantWorkDf = Df
            WantWork = each.shape[0]

    baseDf = pd.DataFrame(list(dict.fromkeys(new_df['HaveWorked'+field] + new_df['WantWork'+c])), columns=[c]).set_index(c)
    baseDf = baseDf.join(HaveWorkedDf).join(WantWorkDf)
    baseDf["Changes"] = baseDf["WantWork"]-baseDf["HaveWorked"]
    baseDf["HaveWorked%"] = baseDf["HaveWorked"]/HaveWorked*100
    baseDf["WantWork%"] = baseDf["WantWork"]/WantWork*100
    new_df[c] = baseDf.sort_values(by=["Changes"], ascending=False)

    display(c)

    display(new_df[c].style.set_properties(**{'background-color': 'black', 'color': 'black', 'border-color': 'white'})\
                     .bar(subset=['Changes'], align='mid', color=['#900C3F', '#649867'])\
                     .background_gradient(subset=['HaveWorked', 'WantWork'], low=0, high=0.5, cmap="BuPu")\
                     .background_gradient(subset=['HaveWorked%', 'WantWork%'], low=0, high=0.5, cmap="RdPu"))



In [11]:
fields = ["Language", "Framework", "Database", "Platform"]

for i in fields:
    techTrend(i)

'Language'

,HaveWorked,WantWork,Changes,HaveWorked%,WantWork%
Language,,,,,
Go,602,2161,1559,4.45069,17.2562
Rust,168,1162,994,1.24205,9.27893
TypeScript,1382,2241,859,10.2174,17.8951
F#,185,892,707,1.36774,7.12289
Swift,832,1536,704,6.15112,12.2654
Haskell,242,918,676,1.78915,7.33051
Scala,518,1187,669,3.82966,9.47856
Elixir,138,636,498,1.02026,5.07866
Clojure,142,515,373,1.04983,4.11243


'Framework'

,HaveWorked,WantWork,Changes,HaveWorked%,WantWork%
Framework,,,,,
React,1552,3290,1738,21.2778,38.7104
Node.js,3394,4189,795,46.5314,49.2882
Xamarin,606,1375,769,8.3082,16.1784
Hadoop,389,1113,724,5.33315,13.0957
.NET Core,2354,3064,710,32.2731,36.0513
Spark,350,931,581,4.79846,10.9542
Firebase,647,968,321,8.8703,11.3896
AngularJS,3217,3505,288,44.1047,41.2401
Cordova,742,727,-15,10.1727,8.55395


'Database'

,HaveWorked,WantWork,Changes,HaveWorked%,WantWork%
Database,,,,,
MongoDB,2191,3387,1196,20.3021,36.98
Redis,1671,2470,799,15.4837,26.968
Cassandra,337,1100,763,3.12268,12.01
PostgreSQL,2919,3129,210,27.0478,34.1631
SQLite,2808,2079,-729,26.0193,22.699
Oracle,1685,951,-734,15.6134,10.3832
SQL Server,4175,2658,-1517,38.6861,29.0206
MySQL,5675,3324,-2351,52.5852,36.2922


'Platform'

,HaveWorked,WantWork,Changes,HaveWorked%,WantWork%
Platform,,,,,
Raspberry Pi,1523,2725,1202,14.3951,26.8261
Android,2650,3621,971,25.0473,35.6468
Amazon Web Services (AWS),3046,3916,870,28.7902,38.5509
iOS,1649,2437,788,15.586,23.9909
Arduino,923,1526,603,8.72401,15.0226
Serverless,255,769,514,2.41021,7.57039
Microsoft Azure,1285,1759,474,12.1456,17.3164
Windows Phone,381,505,124,3.60113,4.97145
Mainframe,144,120,-24,1.36106,1.18133
